In [4]:
import glob 
import numpy as np
import xarray as xr
import pandas as pd 
import seaborn as sns
from scipy import optimize
from scipy import interpolate
import matplotlib.pyplot as plt 

In [5]:

with xr.open_dataset('./link_test/sprg/nc/sprg_2000a_dt_0.5_dx_100_MB_-0.3_OFF_spline_k2.nc') as src: 
    # correct for minimum ice thickness
    src["height"] = xr.where(src.height <= 10, 0, src.height)
    # apply sigma coordinate transform for vertical coordinate
    src["Z"]     = src.zbed + src.Z * src.height 
    # Calculate the magnitude of the velocity vectors
    src['vel_m'] = np.sqrt(src['velocity 1']**2 + src['velocity 2']**2)    
    
Vol = src.height.isel(coord_2=-1).integrate("coord_1") /\
      src.height.isel(coord_2=-1).isel(t=0).integrate("coord_1")



In [8]:
files   = glob.glob('./link_test/sprg/nc/sprg_2000a_dt_0.5_dx_100_MB_-0.*.nc')
xarrays = []
for src_fp in sorted(files): 
    with xr.open_dataset(src_fp) as src: 
        # correct for minimum ice thickness
        src["height"] = xr.where(src.height <= 10, 0, src.height)
        # apply sigma coordinate transform for vertical coordinate
        src["Z"]     = src.zbed + src.Z * src.height 
        # Calculate the magnitude of the velocity vectors
        src['vel_m'] = np.sqrt(src['velocity 1']**2 + src['velocity 2']**2)    
    
    xarrays.append(src)

In [9]:
# Concatenate the .nc files via their mass balance offset
mf_dataset = xr.concat(xarrays,
                       pd.Index(data = np.arange(-0.1,-0.6,-0.1), name='Delta_MB'))

In [ ]:
np.isnan(mf_dataset.isel(Delta_MB=-2).Z)

In [ ]:
len(mf_dataset.isel(Delta_MB=-2).where(~np.isnan(mf_dataset.isel(Delta_MB=-2).Z), drop=True).t)

In [ ]:
fig, ax = plt.subplots(figsize=(9,3))

ax.plot(src.coord_1[::-1]/1000.,
        src.isel(t=0,coord_2=-1).Z,
        color='k', ls=':', lw=1.0, label=r'$z_{\rm s}(t=0)$')


ax.plot(src.coord_1[::-1]/1000.,
        src.isel(t=-7,coord_2=-1).Z,
        color='tab:blue', 
        label=r'$\Delta \dot{b} =$'+'{:.1f}'.format(9.0))

ax.plot(src.coord_1[::-1]/1000.,
        src.isel(t=0,coord_2=-1).zbed,
        color='k')

ax.legend()

# fig.savefig('/Users/andrewnolan/Desktop/Tweeds_Test_Zs.png', dpi=300, bbox_inches='tight', facecolor='w')

In [ ]:
fig, ax = plt.subplots()

ax.plot(Vol.t, Vol)

# ax.legend()
ax.set_ylabel('Relative Volume Per Unit Width')
ax.set_xlabel('Time (years)')

# fig.savefig('/Users/andrewnolan/Desktop/crmpt12-restart.png', dpi=300, bbox_inches='tight', facecolor='w')

In [ ]:
print('Volume(t={:.0f}) : {:.3f}'.format(Vol.t[-1].values, Vol[-1].values))

In [ ]:
fig, ax = plt.subplots()

ax.plot(Vol.t, np.abs(Vol.differentiate('t')))

ax.set_ylabel('$\Delta V / \Delta t$')
ax.set_xlabel('Time (years)')
ax.set_yscale('log')
ax.axhline(1e-6, c='k', ls=":", lw=1)

ax.set_ylim(1e-7, None)
# fig.savefig('/Users/andrewnolan/Desktop/crmpt12-restart-dVdt.png', dpi=300, bbox_inches='tight', facecolor='w')

In [ ]:
plt.plot(src.t, src.vel_m.max(dim=('coord_1', 'coord_2'))*(0.5/100))
#plt.yscale('log')

In [ ]:
plt.imshow(src['zs accumulation flux 2'].isel(coord_2=-1), aspect='auto');
plt.colorbar()

In [ ]:
def calc_AAR(z, b):
    """ z : surface elev|ation    [m a.s.l.]
        b : surface mass balance 
    """
    glac_area  = z.max() - z.min()
    accum_idx  = b>=0
    accum_area = z[accum_idx].max() - z[accum_idx].min()
    
    return accum_area/glac_area

In [ ]:
z = src.Z.isel(coord_2=-1, t=-1).values 
b = src['zs accumulation flux 2'].isel(coord_2=-1, t=-1)

AAR = calc_AAR(z,b)
print(AAR)

In [ ]:
import scipy.interpolate as interpolate


knots = np.loadtxt("../../input_data/mass_balance/KMR_MB_k_2_s_2500_knots.dat")
coefs = np.loadtxt("../../input_data/mass_balance/KMR_MB_k_2_s_2500_coefs.dat")
tck   = (knots, coefs, 2)

z = src.Z.isel(coord_2=-1, t=-1).values
B = interpolate.splev(z, tck) - 0.57 

norm_z    = (z - z.min())/(z.max()-z.min())
accum_idx = b>=0

plt.scatter(norm_z, src['zs accumulation flux 2'].isel(coord_2=-1, t=-1));
plt.plot(norm_z, B, color='k');

plt.axhline(0.0)
plt.axvline(norm_z[accum_idx].min())

In [ ]:
norm_z[accum_idx].min()

In [ ]:
profile_fps = glob.glob('result/*/*.spinup*')

array_list = []
for fp in profile_fps:
    df = pd.read_csv(fp, sep='\t',names=['dx', 'off', 'time'])
    
    header_fp = glob.glob('/'.join(fp.split('/')[:-1]) + "/mesh_dx*/mesh.header")[0]
    n_nodes   = pd.read_csv(header_fp,sep='\s+', header=None)[0][0]
    
    array_list.append(np.stack([np.ones(len(df.time.values))*n_nodes,
                                df.time.values]))

nn_times = np.hstack(array_list)
# df = pd.concat(df_list).reset_index(drop=True)



In [ ]:
plt.scatter(nn_times[0], nn_times[1])

In [ ]:
keys  = [
        # 'twds-b',
        # 'klut-a',
        # 'klut-b',
        'fish']
paths = '../../input_data/topography/{}_{}.dat'

for i, key in enumerate(keys): 
    zb_fp = paths.format(key, 'bed')
    zs_fp = paths.format(key, 'surf')
    
    x   = np.loadtxt(zs_fp)[:,0]
    z_s = np.loadtxt(zs_fp)[:,1]
    z_b = np.loadtxt(zb_fp)[:,1]
    
    
    knots = np.loadtxt("../../input_data/mass_balance/KMR_MB_k_2_s_2500_knots.dat")
    coefs = np.loadtxt("../../input_data/mass_balance/KMR_MB_k_2_s_2500_coefs.dat")
    tck   = (knots, coefs, 2)

    
    B = interpolate.splev(z_s, tck) + 6.5

    # plt.plot(z_s, B)
    plt.plot(x, z_s)
    plt.plot(x, z_b, 'k')
    
    AAR = calc_AAR(z_s,B)
    print(AAR)

In [ ]:
zb_fp

In [ ]:
with xr.open_dataset('./result/sprg/nc/sprg_1000a_dt_1_dx_200_MB_0.9_OFF.nc') as src: 
    # correct for minimum ice thickness
    src["depth"] = xr.where(src.depth <= 10, 0, src.depth)
    # apply sigma coordinate transform for vertical coordinate
    src["Z"]     = src.zbed + src.Z * src.height 
    # Calculate the magnitude of the velocity vectors
    src['vel_m'] = np.sqrt(src['velocity 1']**2 + src['velocity 2']**2)    

In [ ]:
23231 / 60 / 60 